# GPT-3.5-Turbo Model
Creating a question answering chatbot using GPT-3.5. Adapted from: https://github.com/openai/openai-cookbook/blob/main/examples/Question_answering_using_embeddings.ipynb

num_tokens() IS OFTEN USED IN KNOWLEDGE.PY (and other files??) WITHOUT SPECIFYING THE EMBEDDING MODEL.

In [4]:
import sys
sys.path.append("modules")
from modules.config import *
from modules.knowledge import *
from modules.chatbot import *
from modules.embedding_functions import *
from modules.query import *
sys.path.remove("modules")

In [5]:
filename = 'assets/knowledge/Fundamentals_of_Digital_Image_Processing_Textbook.pdf'

In [8]:
# get_embedding("Joel", embedding_model=self.embedding_model)
CompVisionKnowledge = Knowledge('CompVisionPDF', 'GPT')
# for page in WIKI_PAGES:
CompVisionKnowledge.append_wikipedia_page(WIKI_PAGE) # page

The following Wikipedia page has been successfully added to the knowledge database: Computer vision


In [9]:
CompVisionKnowledge.df

,Source,Heading,Subheading,Page,Content,Section,Tokens,Embedding
0,Wikipedia (Computer vision),,,,\nComputer vision tasks include methods for ac...,Wikipedia (Computer vision),301.0,"[-0.017756473273038864, 0.0029741437174379826,..."
1,Wikipedia (Computer vision),Definition,,,\nComputer vision is an interdisciplinary fiel...,Wikipedia (Computer vision)->Definition,158.0,"[-0.0211649127304554, 0.004849217366427183, 0...."
2,Wikipedia (Computer vision),History,,,"\nIn the late 1960s, computer vision began at ...",Wikipedia (Computer vision)->History,507.0,"[-0.011456056497991085, -0.004036989063024521,..."
4,Wikipedia (Computer vision),Related fields,Solid-state physics,,\nSolid-state physics is another field that is...,Wikipedia (Computer vision)->Related fields->S...,120.0,"[0.001918427529744804, 0.011286016553640366, -..."
5,Wikipedia (Computer vision),Related fields,Neurobiology,,\nNeurobiology has greatly influenced the deve...,Wikipedia (Computer vision)->Related fields->N...,293.0,"[-0.009113207459449768, 0.0011424196418374777,..."
6,Wikipedia (Computer vision),Related fields,Signal processing,,\nYet another field related to computer vision...,Wikipedia (Computer vision)->Related fields->S...,103.0,"[-0.027323242276906967, 0.007451199926435947, ..."
7,Wikipedia (Computer vision),Related fields,Robotic navigation,,\nRobot navigation sometimes deals with autono...,Wikipedia (Computer vision)->Related fields->R...,64.0,"[0.0034060052130371332, -0.014033270068466663,..."
8,Wikipedia (Computer vision),Related fields,Other fields,,\nBesides the above-mentioned views on compute...,Wikipedia (Computer vision)->Related fields->O...,119.0,"[0.0024171562399715185, -0.003781835548579693,..."
9,Wikipedia (Computer vision),Related fields,Distinctions,,\nThe fields most closely related to computer ...,Wikipedia (Computer vision)->Related fields->D...,639.0,"[-0.017146369442343712, 0.005803829059004784, ..."
10,Wikipedia (Computer vision),Applications,,,\nApplications range from tasks such as indust...,Wikipedia (Computer vision)->Applications,272.0,"[-0.022551782429218292, 0.005745057016611099, ..."


In [10]:
# CompVisionKnowledge = Knowledge('CompVisionPDF', 'GPT')
CompVisionKnowledge.append_pdf(filename, 'CompVisionPDF')

The following PDF has been successfully added to the knowledge database: CompVisionPDF (assets/knowledge/Fundamentals_of_Digital_Image_Processing_Textbook.pdf)


In [11]:
CompVisionKnowledge.export_to_csv(GPT_KNOWLEDGE_FILENAME)
CompVisionKnowledge.df

,Source,Heading,Subheading,Page,Content,Section,Tokens,Embedding
0,Wikipedia (Computer vision),,,,\nComputer vision tasks include methods for ac...,Wikipedia (Computer vision),301.0,"[-0.017756473273038864, 0.0029741437174379826,..."
1,Wikipedia (Computer vision),Definition,,,\nComputer vision is an interdisciplinary fiel...,Wikipedia (Computer vision)->Definition,158.0,"[-0.0211649127304554, 0.004849217366427183, 0...."
2,Wikipedia (Computer vision),History,,,"\nIn the late 1960s, computer vision began at ...",Wikipedia (Computer vision)->History,507.0,"[-0.011456056497991085, -0.004036989063024521,..."
4,Wikipedia (Computer vision),Related fields,Solid-state physics,,\nSolid-state physics is another field that is...,Wikipedia (Computer vision)->Related fields->S...,120.0,"[0.001918427529744804, 0.011286016553640366, -..."
5,Wikipedia (Computer vision),Related fields,Neurobiology,,\nNeurobiology has greatly influenced the deve...,Wikipedia (Computer vision)->Related fields->N...,293.0,"[-0.009113207459449768, 0.0011424196418374777,..."
...,...,...,...,...,...,...,...,...
110,CompVisionPDF,,,339/340,"\nEdge detection\nchallenges, 270\nfiltering f...",CompVisionPDF-> p.339/340,1581.0,"[-0.002906471025198698, 0.026034852489829063, ..."
111,CompVisionPDF,,,340/341/342,"\nImage\ncolour, 2 3\ncolour spaces, 9 14\nper...",CompVisionPDF-> p.340/341/342,1556.0,"[0.0033323888201266527, 0.030085023492574692, ..."
112,CompVisionPDF,,,343/344,"\nMatlab, 1, 3, 14 17, 19, 35, 52, 56, 59, 61,...",CompVisionPDF-> p.343/344,1557.0,"[-0.0015604019863530993, 0.013584878295660019,..."
113,CompVisionPDF,,,345/346/347/348/349/350/351/352/353/354,"\nResolution, 3 5\nbit resolution, 4\nspatial ...",CompVisionPDF-> p.345/346/347/348/349/350/351/...,1418.0,"[0.02063027210533619, 0.016986483708024025, 0...."


In [29]:
Lec1QAPairs = [
    ["Q: What is this module's name?", "A: Introduction to Image Processing"],
    ["Q: What is this module's code?", "A: COMP 2032"],
    ["Q: When and where are the lectures held?", "A: F3A04 (Block F3, Level A, Room 04) on Wednesdays (09:00 to 11:00)"],
    ["Q: When and where are the labs held?", "A: BB80 (Block B, Level B, Room 80) on Tuesdays (14:00 to 16:00)"],
    ["Q: What sources can I use for wider reading?", "A: Digital Image Processing by R.C. Gonzalez and R.E. Woods. (2018). . (Fourth Edition), Fundamentals of Digital Image Processing: A Practical Approach with Examples in Matlab by Chris Solomon and Toby Breckon (2010), Hypermedia Image Processing Reference (HIPR) at the University of Edinburgh"],
    ["Q: Who is the lecturer for this module?", "A: Dr Tissa Chandesa (Tissa.Chandesa@nottingham.edu.my)"],
    ["Q: Who are the teaching assistants for this module?". "A: Mr Mahmood Haithami (hcxmh1@nottingham.edu.my) and Mr Muhammad Waqas (hcxmw1@nottingham.edu.my)"],
    ["Q: What are the assessments for this module?", "A: Coursework (40% weighting) which comprises of Programming and 2000 Word Report and a 1-hour Written Examination (60%), where you must answer all 3 questions"],
    ["Q: What is image analysis?", "A: Image analysis is concerned with making quantitative measurements on images, using image measurements as a proxy for real-world values"],
    ["Q: What are the limitations of the techniques covered in this module?", "A: Not all solutions can be reached using the covered techniques."],
    ["Q: What can image analysis be used for?", "A: medicine, science, manufacturing, food, textiles"],
    ["Q: Is image processing the same as computer graphics?", "A: No"],
    ["Q: What are some actions that can be performed on digital images using image processing?", "A: Acquire an image, store, manipulate, model, analyze, interpret, and display an image."],
    ["Q: What file types will we use for image processing?", "A: jpeg or png"],
    ["Q: Can image processing be applied to videos?", "A: Yes"],
    ["Q: What's the primary aim of this module?", "A: Introduce students to the fundamentals of digital image processing theory and practice by gaining practical experience in writing programs to manipulate digital images. It lays down the foundation for studying advanced topic in related fields"],
    ['Q: What are pictures represented by?', "A: Pixels"],
    ['Q: What colours are used in the greyscale plane?', "A: It is a range from white to black"],
    ['Q: What three colours are used to construct an image?', "A: Red, green and blue"],
    ["Q: What are the three components of image processing?", "A: Image formation, acquisition, colour representation"],
    ['Q: What is image compression used for?', "A: To efficiently represent image data for storage and communication, to minimise disk space and network bandwidth"],
    ["Q: What is the effect of image compression", "A: It reduces the image size but makes the quality poorer and more blurry"],
    ["Q: What is image manipulation used for?", "A: Image manipulation is used to remove noise, sharpen, sharpen and enhance or change the contrast and general appearance of an image."],
    ["Q: What are some examples of image compression formats?", "A: Examples include GIF, JPEG, and PNG."],
    ["Q: What are superpixels?", "A: A grouping of similar pixels used as an intermediate image representation to reduce the number of pixels"],
    ["Q: What technique can be used to find geometric objects?" "A: The Hough Transform"],
    ["Q: What are spatial domain methods?", "A: Methods which operate directly on the image, such as point operations and area operations."],
    ["Q: What is the purpose of image segmentation?", "A: Image segmentation is used to extract specific objects or regions from an image."],
    ["Q: What is the frequency domain method in image processing?", "A: Frequency domain methods involve computing and modifying the power spectrum of an image."],
    ["Q: What is the importance of geometric operations in image processing?", "A: Geometric operations allow for changes in the image's array structure, and include the manipulation of the orientation, rotation, and scaling."],
    ["Q: What is content-based image retrieval?", "A: Content-based image retrieval is a technique for searching large image databases based on their visual content."],
    ["Q: What is painterly rendering in image processing?", "A: Painterly rendering involves processing an image to give it the appearance of a painting, based on the work of a particular artist or movement (e.g. impressionism)."],
    ["Q: What are interactive tools and compositing in image processing?", "A: Interactive tools and compositing involve overlaying and combining multiple images into a single output image."],
    ["Q: What are some programming languages commonly used for image processing?", "A: Commonly used languages include MATLAB, Python (with libraries like PIL and OpenCV), and Java."],
]

Lec2QAPairs = [
    ["Q: What are the two important processes in digital image formation?", "A: Sampling and quantization."],
    ["Q: What is sampling in digital image formation?", "A: Sampling is the process of digitizing the spatial coordinates of an image."],
    ["Q: What is quantization in digital image formation?", "A: Quantization is the process of digitizing the light intensity function of an image."],
    ["Q: What is aliasing and what causes it?", "A: Aliasing is an artifact that occurs when the sampling rate is insufficient, causing the image to become unrecognizable. It is caused by undersampling or sampling at a rate below the Nyquist rate."],
    ["Q: How many pixels are in an image?", "A: The number of pixels in an image depends on its resolution and size. It can vary from image to image."],
    ["Q: How many samples should you take from an image?", "A: Samples must be taken at a rate that is twice the frequency of the highest frequency component to be reconstructed"],
    ["Q: What is the Nyquist Rate?", "A: The minimum sampling rate required to accurately reconstruct an image from its sampled version"],
    ["A: What is under-sampling?", "A: Sampling at a rate which is too course, i.e. below the Nyquist Rate."],
    ["Q: What causes Aliasing?", "A: Under-sampling."],
    ['Q: What is unsampling', "A: The process of reconstructing an image by interpolating pixel values from the sample values"],
    ["Q: What is super resolution?", "A: Super resolution methods involve combining multiple exposures of the same scene to enhance the resolution and quality of an image."],
    ['Q: What is a Bayer pattern?', "A: An array of red, green, and blue color filters arranged in a specific repeating pattern."],
    ['Q: Out of red, green and blue, which colour are our eyes drawn to most?', "A: Green."],
    ["Q: What is color intensity in an image?", "A: Color intensity refers to the level of intensity or brightness of a color in an image. It determines the perceived lightness or darkness of a color."],
    ["Q: How can aliasing be avoided in image sampling?", "A: Aliasing can be avoided in image sampling by ensuring that the sampling rate is at least twice the frequency of the highest frequency component to be reconstructed."],
    ["Q: How does interpolation help in image sampling?", "A: Interpolation helps in image sampling by estimating unknown pixel values based on known neighboring pixel values, filling in the gaps and producing a smoother representation of the image."],
    ["Q: Can gray level resolution be increased in a single image?", "A: No"],
    ['Q: Why is greyscale used in image processing', "A: It makes processing easier, reduces the amount of information in the image, and makes some of the theory simpler."],
    ['Q: How can you convert between colour and greyscale images?', "A: By using a weighted average of the red, green, and blue components, with a higher weighting on green (30% red, 59% green, and 11% blue)."],
    ["Q: What is HSV color space?", "A: HSV color space stands for hue, saturation, and value. It is a color model that represents colors based on their hue (the dominant wavelength), saturation (purity of color), and value (brightness). It is based on colour rather than light."],
    ["Q: Is HSV more or less sensitive to illumination changes relative to RGB?", "A: Less sensitive"],
    ['Q: What two components of HSV is human skin most captured by, out of hue, saturation, and value? ', "A: Human skin is most captured by the hue and saturation components of HSV."],

    ["Q: What are the main components of image processing?", "A: The main components of image processing include image enhancement, image restoration, image compression, image analysis, and image recognition."],
    ["Q: What is the purpose of image enhancement?", "A: The purpose of image enhancement is to improve the visual quality of an image by adjusting its contrast, brightness, sharpness, or removing noise or artifacts."],
    ["Q: What is image restoration?", "A: Image restoration involves the process of recovering an image"]
]

,Source,Heading,Subheading,Content
0,Comp Vision Stuff,{0},NaN,\nGLOBAL \nEDITION\n
0,Comp Vision Stuff,{0},NaN,\nDigital Image Processing\n
0,Comp Vision Stuff,{0},NaN,\n FOURTH EDITION\n
0,Comp Vision Stuff,{0},NaN,\n Rafael C. Gonzalez * Richard E. Woods\n
0,Comp Vision Stuff,{0},NaN,\nwww.EBooksWorld.ir\n
...,...,...,...,...
0,Comp Vision Stuff,{1021},NaN,\n18 Chapter 1 Introduction\n
0,Comp Vision Stuff,{1021},NaN,\n1.1 WHAT IS DIGITAL IMAGE PROCESSING? \n
0,Comp Vision Stuff,{1021},NaN,\nAn image may be defined as a two-dimensional...
0,Comp Vision Stuff,{1021},NaN,\n1.1\n


In [ ]:
output_file_folder = 'assets/'

In [5]:
target_url = 'https://homepages.inf.ed.ac.uk/rbf/HIPR2/median.htm'
import urllib.request  # the lib that handles the url stuff

for line in urllib.request.urlopen(target_url):
    print(line.decode('utf-8')) #utf-8 or iso8859-1 or whatever the page encoding scheme is

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1129)>

In [20]:
CompVisionKnowledgeBERT = Knowledge(WIKI_PAGE, 'BERT')
CompVisionKnowledgeBERT.append_wikipedia_page(WIKI_PAGE)
# save document chunks and embeddings
CompVisionKnowledgeBERT.export_to_csv(BERT_KNOWLEDGE_FILENAME)
CompVisionKnowledgeBERT.df

,Source,Heading,Subheading,Content,Section,Tokens,Embedding
0,Wikipedia (Computer vision),,,\nComputer vision tasks include methods for ac...,Wikipedia (Computer vision),290,"[-0.5566069483757019, 0.6151323318481445, 0.70..."
1,Wikipedia (Computer vision),Definition,,\nComputer vision is an interdisciplinary fiel...,Wikipedia (Computer vision)->Definition,162,"[-0.18940897285938263, 0.5564344525337219, 0.5..."
2,Wikipedia (Computer vision),History,,"\nIn the late 1960s, computer vision began at ...",Wikipedia (Computer vision)->History,243,"[-0.5624328255653381, 0.35494446754455566, 0.6..."
3,Wikipedia (Computer vision),History,,"\nBy the 1990s, some of the previous research ...",Wikipedia (Computer vision)->History,264,"[-0.8420819044113159, 0.011862404644489288, 0...."
5,Wikipedia (Computer vision),Related fields,Solid-state physics,\nSolid-state physics is another field that is...,Wikipedia (Computer vision)->Related fields->S...,123,"[-0.1766018569469452, 0.5509802103042603, 0.11..."
6,Wikipedia (Computer vision),Related fields,Neurobiology,\nNeurobiology has greatly influenced the deve...,Wikipedia (Computer vision)->Related fields->N...,296,"[0.07454751431941986, 0.42800015211105347, 0.1..."
7,Wikipedia (Computer vision),Related fields,Signal processing,\nYet another field related to computer vision...,Wikipedia (Computer vision)->Related fields->S...,106,"[-0.1562240570783615, 0.18830031156539917, 0.4..."
8,Wikipedia (Computer vision),Related fields,Robotic navigation,\nRobot navigation sometimes deals with autono...,Wikipedia (Computer vision)->Related fields->R...,65,"[0.09209920465946198, 0.5207788944244385, 1.26..."
9,Wikipedia (Computer vision),Related fields,Other fields,\nBesides the above-mentioned views on compute...,Wikipedia (Computer vision)->Related fields->O...,122,"[-0.34635502099990845, 0.12120083719491959, 0...."
10,Wikipedia (Computer vision),Related fields,Distinctions,\nThe fields most closely related to computer ...,Wikipedia (Computer vision)->Related fields->D...,244,"[-0.011814514175057411, 0.9892112612724304, 0...."


# Search
Now we'll define a search function that:

Takes a user query and a dataframe with text & embedding columns
Embeds the user query with the OpenAI API
Uses distance between query embedding and text embeddings to rank the texts
Returns two lists:
The top N texts, ranked by relevance
Their corresponding relevance scores

In [8]:
# Todo:
# I need to make it more efficient on the number of tokens
# Check num_tokens usage and specifying the encoding model
# Fix the potential issue fo GPT sections being longer than 1024 tokens when using BART

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.

KeyboardInterrupt



In [13]:
CompVisionGPT = ChatBot("Computer Vision", 'assets/' + GPT_KNOWLEDGE_FILENAME)
print(Query.ask('What is PCA?', CompVisionGPT, show_source=True))

C:\Users\point\OneDrive\Documents\Standard Files\University\Masters\Diss\MSc-Diss\code\modules\query.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.knowledge_used['Index'] = np.arange(len(self.knowledge_used)) + 1
C:\Users\point\OneDrive\Documents\Standard Files\University\Masters\Diss\MSc-Diss\code\modules\query.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.knowledge_used['Cumulative_tokens'] = self.knowledge_used['Tokens'].cumsum()
C:\Users\point\OneDrive\Documents\Standard Files\Un

PCA stands for Principal Component Analysis, which is a statistical technique used for dimensionality reduction of correlated data. It involves finding a new set of uncorrelated variables (or axes) that are linear combinations of the original variables, and which capture the maximum amount of variance in the data. PCA is commonly used in digital imaging for tasks such as data compression, automated facial recognition, and determination of the orientation of basic shapes.

To construct this answer, I used the following documents : 

1. CompVisionPDF-> p.265/266/267:
The reader is probably familiar with the common saying that goes something along the
lines of 'Why ...

2. CompVisionPDF-> p.268/269/270:
For a 2-D space, we must stop here (there are no more dimensions left). Figure 9.9
shows our exampl...

Total tokens used: 3227


C:\Users\point\OneDrive\Documents\Standard Files\University\Masters\Diss\MSc-Diss\code\modules\query.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.knowledge_used['Output'] = '\n\n' + self.knowledge_used['Index'].astype(str) + '. ' + self.knowledge_used[


In [13]:
CompVisionBERT = ChatBot("Computer Vision", 'assets/' + BERT_KNOWLEDGE_FILENAME)
print(Query.ask_bert('When did universities begin teaching Computer Vision?', CompVisionBERT))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

ValueError: operands could not be broadcast together with shapes (768,) (1536,) 

In [20]:
CompVisionGPT = ChatBot("Computer Vision", 'assets/' + GPT_KNOWLEDGE_FILENAME)
print(Query.ask_bart('When did Universities begin teaching Computer Vision?', CompVisionGPT, show_source=True))  # What if the GPT knowledge sections are longer than 1024 tokens?? Need to account for this!

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [19]:
import torch

def describe(x):
    print("Type: {}".format(x.type()))
    print("Shape/size: {}".format(x.shape))
    print("Values: \n{}".format(x))

describe(torch.Tensor(2, 3))

Type: torch.FloatTensor
Shape/size: torch.Size([2, 3])
Values: 
tensor([[6.7410e+22, 2.6729e+23, 5.3689e-05],
        [1.3542e-05, 5.2905e-08, 6.7942e-07]])


# Tasks
Clean the parsed text so we have pure english words

In [31]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

from nltk.tokenize import word_tokenize
text = "It's Here’s to the crazy ones, the misfits, the rebels, the troublemakers, the round pegs in the square holes. The ones who see things differently — they’re not fond of rules. You can quote them, disagree with them, glorify or vilify them, but the only thing you can’t do is ignore them because they change things. They push the human race forward, and while some may see them as the crazy ones, we see genius, because the ones who are crazy enough to think that they can change the world, are the ones who do."
tokens = word_tokenize(text)

# Remove stopwords
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))
filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
print(filtered_tokens)

["'s", '’', 'crazy', 'ones', ',', 'misfits', ',', 'rebels', ',', 'troublemakers', ',', 'round', 'pegs', 'square', 'holes', '.', 'ones', 'see', 'things', 'differently', '—', '’', 'fond', 'rules', '.', 'quote', ',', 'disagree', ',', 'glorify', 'vilify', ',', 'thing', '’', 'ignore', 'change', 'things', '.', 'push', 'human', 'race', 'forward', ',', 'may', 'see', 'crazy', 'ones', ',', 'see', 'genius', ',', 'ones', 'crazy', 'enough', 'think', 'change', 'world', ',', 'ones', '.']


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\point\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\point\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [150]:
sentences = [
"Artificial intelligence (AI) is revolutionising industries and transforming the way we live and work.",
"Machine learning, a subset of AI, enables computers to learn and make predictions or decisions without being explicitly programmed.",
"AI algorithms can analyse vast amounts of data to uncover patterns, detect anomalies, and extract valuable insights.",
"Natural language processing (NLP) allows machines to understand and process human language, facilitating communication between humans and computers.",
"Computer vision enables machines to interpret and understand visual information, enabling applications such as image recognition and object detection.",
"AI-powered virtual assistants like Siri, Alexa, and Google Assistant are becoming increasingly common, providing voice-based interactions and personalised experiences.",
"AI is driving advancements in healthcare, including disease diagnosis, drug discovery, and personalised medicine, leading to improved patient outcomes.",
"In the field of autonomous vehicles, AI plays a crucial role in enabling self-driving cars to perceive their surroundings and make informed decisions.",
"AI is transforming the customer service industry by utilising chatbots and automated systems to provide faster and more efficient support to customers.",
"Ethical considerations, such as transparency, fairness, and privacy, are essential in the development and deployment of AI systems to ensure responsible and accountable use.",
"Cars have revolutionised transportation, providing a convenient and efficient means of travel for people around the world.",
"Automobile manufacturing involves a complex process of designing, engineering, and assembling various components to create a functional vehicle.",
"Safety features such as seat belts, airbags, and anti-lock braking systems have greatly improved the overall safety of cars.",
"Electric vehicles (EVs) are gaining popularity as eco-friendly alternatives to traditional petrol-powered cars, reducing carbon emissions and dependence on fossil fuels.",
"Advanced driver-assistance systems (ADAS) enhance car safety by incorporating technologies like adaptive cruise control and lane-keeping assist.",
"Sports cars are known for their high-performance capabilities, offering speed, agility, and an exhilarating driving experience.",
"Classic cars hold a special place in automotive history, with their timeless designs and nostalgic appeal capturing the hearts of car enthusiasts.",
"Car customisation allows owners to personalise their vehicles, from unique paint jobs and body modifications to performance upgrades.",
"Car-sharing services and ride-hailing apps have transformed the way people access transportation, providing convenient alternatives to car ownership.",
"The future of cars is expected to bring autonomous vehicles, where cars can navigate and operate without human intervention, promising increased safety and efficiency."
]

In [152]:
import gensim
from gensim.models import Word2Vec
from tqdm import tqdm

def preprocess_text(text: str, remove_stopwords: bool) -> str:
    """Function that cleans the input text by going to:
    - remove links
    - remove special characters
    - remove numbers
    - remove stopwords
    - convert to lowercase
    - remove excessive white spaces
    Arguments:
        text (str): text to clean
        remove_stopwords (bool): whether to remove stopwords
    Returns:
        str: cleaned text
    """
    # remove links
    text = re.sub(r"http\S+", "", text)
    # remove numbers and special characters
    text = re.sub("[^A-Za-z]+", " ", text)
    # remove stopwords
    if remove_stopwords:
        # 1. create tokens
        tokens = nltk.word_tokenize(text)
        # 2. check if it's a stopword
        tokens = [w.lower().strip() for w in tokens if not w.lower() in stopwords.words("english")]
        # return a list of cleaned tokens
        return tokens

tokenised_sentences = [preprocess_text(sentence, remove_stopwords=True) for sentence in sentences]

In [153]:
tokenised_sentences

[['artificial',
  'intelligence',
  'ai',
  'revolutionising',
  'industries',
  'transforming',
  'way',
  'live',
  'work'],
 ['machine',
  'learning',
  'subset',
  'ai',
  'enables',
  'computers',
  'learn',
  'make',
  'predictions',
  'decisions',
  'without',
  'explicitly',
  'programmed'],
 ['ai',
  'algorithms',
  'analyse',
  'vast',
  'amounts',
  'data',
  'uncover',
  'patterns',
  'detect',
  'anomalies',
  'extract',
  'valuable',
  'insights'],
 ['natural',
  'language',
  'processing',
  'nlp',
  'allows',
  'machines',
  'understand',
  'process',
  'human',
  'language',
  'facilitating',
  'communication',
  'humans',
  'computers'],
 ['computer',
  'vision',
  'enables',
  'machines',
  'interpret',
  'understand',
  'visual',
  'information',
  'enabling',
  'applications',
  'image',
  'recognition',
  'object',
  'detection'],
 ['ai',
  'powered',
  'virtual',
  'assistants',
  'like',
  'siri',
  'alexa',
  'google',
  'assistant',
  'becoming',
  'increasing

In [154]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
lemmatizer = WordNetLemmatizer()

from nltk.corpus import wordnet

# function to convert nltk tag to wordnet tag
def get_wordnet_pos(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN # Default to noun if the POS tag is unknown/ambiguous

def lemmatize_sentence(tokenised_sentence):
    tagged_tokens = nltk.pos_tag(tokenised_sentence)
    lemmatized_words = []
    for word, tag in tagged_tokens:
        pos = get_wordnet_pos(tag)
        lemmatized_words.append(lemmatizer.lemmatize(word, pos=pos))
    return lemmatized_words

# refined_sentences = [sentence.strip()]
lemmatised_sentences = [lemmatize_sentence(s) for s in tokenised_sentences]
lemmatised_sentences

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\point\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\point\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\point\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


[['artificial',
  'intelligence',
  'ai',
  'revolutionise',
  'industry',
  'transform',
  'way',
  'live',
  'work'],
 ['machine',
  'learn',
  'subset',
  'ai',
  'enable',
  'computer',
  'learn',
  'make',
  'prediction',
  'decision',
  'without',
  'explicitly',
  'program'],
 ['ai',
  'algorithm',
  'analyse',
  'vast',
  'amount',
  'data',
  'uncover',
  'pattern',
  'detect',
  'anomaly',
  'extract',
  'valuable',
  'insight'],
 ['natural',
  'language',
  'processing',
  'nlp',
  'allow',
  'machine',
  'understand',
  'process',
  'human',
  'language',
  'facilitate',
  'communication',
  'human',
  'computer'],
 ['computer',
  'vision',
  'enable',
  'machine',
  'interpret',
  'understand',
  'visual',
  'information',
  'enable',
  'application',
  'image',
  'recognition',
  'object',
  'detection'],
 ['ai',
  'power',
  'virtual',
  'assistant',
  'like',
  'siri',
  'alexa',
  'google',
  'assistant',
  'become',
  'increasingly',
  'common',
  'provide',
  'voice'

In [155]:
from gensim.models import Word2Vec

VECTOR_SIZE = 100
MIN_COUNT = 1
WINDOW = 3
SG = 1

model = Word2Vec(
    sentences=lemmatised_sentences,
    vector_size=VECTOR_SIZE,
    min_count=MIN_COUNT,
    sg=SG
)

model = Word2Vec(sentences=lemmatised_sentences, min_count=1)

In [157]:
model.wv.similarity('ai', 'algorithm')

# model.wv.most_similar(positive=['ai'], topn=5)

-0.052282296

In [107]:
from sklearn.manifold import TSNE
def reduce_dimensions(model):
    num_components = 2  # number of dimensions to keep after compression

    # extract vocabulary from model and vectors in order to associate them in the graph
    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    # apply TSNE
    tsne = TSNE(n_components=num_components, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels


def plot_embeddings(x_vals, y_vals, labels):
    import plotly.graph_objs as go
    fig = go.Figure()
    trace = go.Scatter(x=x_vals, y=y_vals, mode='markers', text=labels)
    fig.add_trace(trace)
    fig.update_layout(title="Word2Vec - Visualising embeddings with TSNE")
    fig.show()
    return fig


x_vals, y_vals, labels = reduce_dimensions(model)

plot = plot_embeddings(x_vals, y_vals, labels)

C:\Users\point\anaconda3\lib\site-packages\sklearn\manifold\_t_sne.py:780: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

C:\Users\point\anaconda3\lib\site-packages\sklearn\manifold\_t_sne.py:790: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



In [158]:
from transformers import AutoTokenizer, GPT2Model
import torch

tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = GPT2Model.from_pretrained("gpt2")

inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
outputs = model(**inputs)

last_hidden_states = outputs.last_hidden_state